# MVD 10. cvičení
V dnešním cvičení rozšíříme BERT model o jednu FCNN vrstvu, která se dotrénuje na úloze klasifikace sentimentu. Bude se vycházet z předtrénovaného BERT modelu a přidaná vrstva se bude trénovat s Pytorch knihovnou.

In [1]:
import random
import torch
import numpy as np
import pandas as pd

In [2]:
# Parametry trénování
epochs = 1
lr = 1e-6
batch_size = 2

## 1. část - Příprava dat pro detekci sentimentu

Stáhněte si data filmových recenzí na [Kaggle datasets](https://www.kaggle.com/columbine/imdb-dataset-sentiment-analysis-in-csv-format) a rozbalte je do adresáře data. Spusťte a projděte následující kód.

In [3]:
# Vybrána menší množina dat -> lze spustit i na cpu
train_df = pd.read_csv('data/Train.csv').head(1000)
train_df['type'] = 'train'
test_df = pd.read_csv('data/Test.csv').head(100)
test_df['type'] = 'test'

data_df = pd.concat([train_df, test_df])

print(data_df.shape)
print(data_df.head())
print(data_df.text.iloc[0])
# Kontrola vyvážení datasetu
print(data_df.label.value_counts())

(1100, 3)
                                                text  label   type
0  I grew up (b. 1965) watching and loving the Th...      0  train
1  When I put this movie in my DVD player, and sa...      0  train
2  Why do people who do not know what a particula...      0  train
3  Even though I have great interest in Biblical ...      0  train
4  Im a die hard Dads Army fan and nothing will e...      1  train
I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of t

In [4]:
# Nastavení random seedu pro možnost reprodukce experimentu
seed = 42
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\cuda\__init__.py:88: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
encoder_train = tokenizer.batch_encode_plus(data_df[data_df['type']=='train'].text.values,
                                           add_special_tokens = True,
                                           padding = True,
                                           truncation=True,
                                           return_tensors = 'pt')

encoder_test = tokenizer.batch_encode_plus(data_df[data_df['type']=='test'].text.values,
                                           add_special_tokens = True,
                                           padding = True,
                                           truncation=True,
                                           return_tensors = 'pt')

input_ids_train = encoder_train['input_ids']
attention_masks_train = encoder_train["attention_mask"]
labels_train = torch.tensor(data_df[data_df['type'] == 'train'].label.values)

input_ids_test = encoder_test['input_ids']
attention_masks_test = encoder_test["attention_mask"]
labels_test = torch.tensor(data_df[data_df['type'] == 'test'].label.values)

### Attention masks
Attention masky používáme z důvodu paddingu (doplnění nul do určité délky). Attention maska obsahuje 0 nebo 1 a říká modelu, kde se nachází původní vstup a kde padding, na který se nemá zaměřovat. 


In [6]:
from torch.utils.data import RandomSampler, SequentialSampler, DataLoader

data_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
data_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

dataloader_train = DataLoader(data_train, shuffle = True, batch_size = batch_size)
dataloader_test = DataLoader(data_test, batch_size = batch_size)

## 2. část - Příprava modelu a trénování

In [7]:
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

optimizer = AdamW(model.parameters(), lr = lr)
scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps = 0,
                num_training_steps = len(dataloader_train)*epochs 
            )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### BertForSequenceClassification
Tato třída již obsahuje jednu přidanou lineární vrstvu se 2 výstupními třídami (viz parametr num_labels). Tuto vrstvu uvidíte na konci seznamu vrstev.

In [8]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
print("Device: {}".format(device))
if(torch.cuda.is_available()):
    torch.cuda.empty_cache()
model.to(device)

Device: cpu


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
from tqdm.notebook import tqdm
torch.cuda.empty_cache()
for epoch in tqdm(range(epochs)):
    model.train()
    train_loss = 0
    print("train")
    with tqdm(dataloader_train, desc = f'Epoch: {epoch}') as pbar:
        pbar.set_postfix({'training_loss':f'{0}'})
        for batch in pbar:
            model.zero_grad()

            inputs = {
                "input_ids" : batch[0].to(device),
                "attention_mask" : batch[1].to(device),
                "labels" : batch[2].to(device)
            }
            outputs = model(**inputs)

            loss = outputs[0]
            # outputs[1] -> logits
            train_loss += loss.item()
            loss.backward()

            optimizer.step()
            scheduler.step()

            pbar.set_postfix({'training_loss':f'{(loss.item() / len(batch)):.3f}'})
            print({'training_loss':f'{(loss.item() / len(batch)):.3f}'})
    loss_train_avg = train_loss / len(dataloader_train)
    tqdm.write(f'Training Loss: {loss_train_avg}')

  0%|          | 0/1 [00:00<?, ?it/s]

train


Epoch: 0:   0%|          | 0/500 [00:00<?, ?it/s]

{'training_loss': '0.245'}
{'training_loss': '0.189'}
{'training_loss': '0.295'}
{'training_loss': '0.242'}
{'training_loss': '0.168'}
{'training_loss': '0.228'}
{'training_loss': '0.221'}
{'training_loss': '0.143'}
{'training_loss': '0.333'}
{'training_loss': '0.222'}
{'training_loss': '0.243'}
{'training_loss': '0.294'}
{'training_loss': '0.146'}


KeyboardInterrupt: 

## 3. Část - Vyhodnocení
Vytvořte funkci `eval`, která bude přijímat jako vstupní parametr test (nebo valid) DataLoader. V této funkci projdete data v evaluačním módu modelu, spočítáte loss na daných datech a využijete výstup `output[1] # logits` pro výpočet úspěšnosti (accuracy) a f1-skóre (pomocí sklearn funkce).

In [24]:
from sklearn.metrics import f1_score 
import torch.nn.functional as fun
from torch.utils.data import DataLoader
import numpy as np

def eval(data:DataLoader): 
    print(len(data))
    model.eval()
    num = 0
    hits = 0    
    lossAvg = 0
    scoreAvg = 0
    for batch in data:
        inputs = {
            "input_ids" : batch[0].to(device),
            "attention_mask" : batch[1].to(device),
            "labels" : batch[2].to(device)
        }
        outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        #print("loss",loss.detach().numpy())
        #print("labels",inputs["labels"])
        #print("logits",logits)
        a = inputs["labels"].detach().numpy()
        b = logits.detach().numpy()
        #print(logits.shape)
        probabilities = fun.softmax(logits, dim=-1).detach().numpy()
        #print(probabilities)
        pred = []
        for i in probabilities:
            if(i[0]>i[1]):
                pred.append(0)
            else:
                pred.append(1)
        pred = np.asarray(pred)
        num+=2
        hits+= np.sum(pred==a)
        loss = np.sum(loss.detach().numpy())
        lossAvg = lossAvg + ((loss - lossAvg) / (num/2))
        scoreAvg = scoreAvg + ((f1_score(a,pred) - scoreAvg)/(num/2))
        #print("f1_score",f1_score(a,pred))
        print("acuraci",hits/num)
        print("lossAvg",lossAvg)
        print("scoreAvg",lossAvg)
        #ids,ign,label = outputs
    print("acuraci",hits/num)
    pass

eval(dataloader_test)


#model.eval()

50
acuraci 0.0
lossAvg 0.9670160412788391
scoreAvg 0.9670160412788391
acuraci 0.25
lossAvg 0.8507828414440155
scoreAvg 0.8507828414440155
acuraci 0.3333333333333333
lossAvg 0.7998071114222208
scoreAvg 0.7998071114222208
acuraci 0.375
lossAvg 0.7665020674467087
scoreAvg 0.7665020674467087
acuraci 0.3
lossAvg 0.797487449645996
scoreAvg 0.797487449645996
acuraci 0.3333333333333333
lossAvg 0.7836709221204121
scoreAvg 0.7836709221204121
acuraci 0.2857142857142857
lossAvg 0.8142510652542113
scoreAvg 0.8142510652542113
acuraci 0.25
lossAvg 0.8358660638332366
scoreAvg 0.8358660638332366


KeyboardInterrupt: 